In [45]:
## imports
import pandas as pd
import numpy as np
#import plotnine
#from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [5]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

In [17]:
df.report_dt

AttributeError: 'Series' object has no attribute 'month'

## Warm-up Demo

In [7]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 1.01 s, sys: 9.97 ms, total: 1.02 s
Wall time: 1.02 s


In [21]:
%%time
for i,r in df.iterrows():
    r.X + r.Y
    print(i)
    if i == 6:
        break

0
1
2
3
4
5
6
CPU times: user 29.3 ms, sys: 20.9 ms, total: 50.2 ms
Wall time: 71.8 ms


In [23]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 221 ms, sys: 20.5 ms, total: 242 ms
Wall time: 265 ms


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

In [25]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 618 μs, sys: 141 μs, total: 759 μs
Wall time: 699 μs


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

# Practice

In [35]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


In [31]:
df.CCN

0        20118678
1        20023507
2        20090091
3        20048262
4        20050930
           ...   
27926    20013962
27927    20015078
27928    20157380
27929    20024376
27930    20089061
Name: CCN, Length: 27931, dtype: int64

**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [47]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 279     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         1308   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         4675        DAY  OTHERS         THEFT F/AUTO   
         11621   EVENING  OTHERS          THEFT/OTHER   
         15616       DAY  OTHERS          THEFT/OTHER   

                                                     BLOCK    XBLOCK  \
20165648 279            1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         1308             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         4675          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         11621  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15616   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 279    137478.0  ...       10603.0    Precinct 144  38.905159   
         1308   135959.0  ...        8200.0     Precinct 89  38.891475   
         4675   137456.0  ...       10602.0     Precinct 83  38.904961   
         11621  136899.0  ...        5900.0    Precinct 143  38.899942   
         15616  135255.0  ...        6500.0     Precinct 89  38.885133   

                LONGITUDE           BID              START_DATE  \
20165648 279   -77.005891          NOMA  2020/11/20 20:15:26+00   
         1308  -76.999516           NaN  2020/11/20 03:02:27+00   
         4675  -76.997314           NaN  2020/11/19 22:30:39+00   
         11621 -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         15616 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 279    2020/11/20 21:46:24+00  596093043             NaN   
         1308                      NaN  596105470             NaN   
         4675   2020/11/20 03:00:43+00  596362501             NaN   
         11621  2020/11/20 22:08:28+00  596387673             NaN   
         15616                     NaN  596402829             NaN   

                               report_dt  
20165648 279   2020-11-20 22:17:27+00:00  
         1308  2020-11-20 04:27:36+00:00  
         4675  2020-11-20 15:06:04+00:00  
         11621 2020-11-20 22:07:10+00:00  
         15616 2020-11-20 12:46:32+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [49]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 734    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014      DAY  OTHERS         THEFT F/AUTO   
         14015      DAY  OTHERS         THEFT F/AUTO   
         18683      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 734         1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         9297        1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   
         14014       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         14015  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         18683       700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 734    ...       10700.0     Precinct 17  38.905648 -77.039822   
         9297   ...        5802.0    Precinct 129  38.897346 -77.027563   
         14014  ...        5303.0     Precinct 15  38.909638 -77.040089   
         14015  ...        4100.0     Precinct 13  38.913346 -77.050526   
         18683  ...        5801.0    Precinct 129  38.899121 -77.021926   

                            BID              START_DATE  \
20123250 734    GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         9297          DOWNTOWN  2020/08/29 23:08:57+00   
         14014              NaN  2020/08/28 22:00:23+00   
         14015              NaN  2020/08/26 22:00:29+00   
         18683         DOWNTOWN  2020/08/29 16:05:40+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 734    2020/08/29 19:00:05+00  596095073             NaN   
         9297                      NaN  596376108             NaN   
         14014  2020/08/29 08:00:27+00  596397251             NaN   
         14015  2020/08/27 12:00:51+00  596397252             NaN   
         18683  2020/08/29 16:08:33+00  596447637             NaN   

                               report_dt  
20123250 734   2020-08-29 22:04:46+00:00  
         9297  2020-08-30 00:05:52+00:00  
         14014 2020-08-29 16:05:18+00:00  
         14015 2020-08-29 16:45:57+00:00  
         18683 2020-08-29 17:15:19+00:00  

[5 rows x 26 columns]

In [53]:
def find_crime_matches(C_Tar, C_Oth, time_window_minutes=20):
    # Create a time window
    C_Tar['time_window_start'] = C_Tar['report_dt']
    C_Tar['time_window_end'] = C_Tar['report_dt'] + pd.Timedelta(minutes=time_window_minutes)

    # Merge the two DataFrames on the 'WARD' column
    merged = pd.merge(C_Tar, C_Oth, on='WARD', suffixes=('_tar', '_oth'))

    # Filter merged DataFrame based on the time window
    matches = merged[(merged['report_dt_oth'] >= merged['time_window_start']) & 
                     (merged['report_dt_oth'] <= merged['time_window_end'])]

    # Select relevant columns and add matched CCN
    matches = matches[['CCN_tar', 'WARD', 'report_dt_oth']].rename(columns={'CCN_tar': 'matched_CCN'})
    
    return matches.reset_index(drop=True)

all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 734    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014      DAY  OTHERS         THEFT F/AUTO   
         14015      DAY  OTHERS         THEFT F/AUTO   
         18683      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 734         1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         9297        1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   
         14014       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         14015  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         18683       700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 734    ...       10700.0     Precinct 17  38.905648 -77.039822   
         9297   ...        5802.0    Precinct 129  38.897346 -77.027563   
         14014  ...        5303.0     Precinct 15  38.909638 -77.040089   
         14015  ...        4100.0     Precinct 13  38.913346 -77.050526   
         18683  ...        5801.0    Precinct 129  38.899121 -77.021926   

                            BID              START_DATE  \
20123250 734    GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         9297          DOWNTOWN  2020/08/29 23:08:57+00   
         14014              NaN  2020/08/28 22:00:23+00   
         14015              NaN  2020/08/26 22:00:29+00   
         18683         DOWNTOWN  2020/08/29 16:05:40+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 734    2020/08/29 19:00:05+00  596095073             NaN   
         9297                      NaN  596376108             NaN   
         14014  2020/08/29 08:00:27+00  596397251             NaN   
         14015  2020/08/27 12:00:51+00  596397252             NaN   
         18683  2020/08/29 16:08:33+00  596447637             NaN   

                               report_dt  
20123250 734   2020-08-29 22:04:46+00:00  
         9297  2020-08-30 00:05:52+00:00  
         14014 2020-08-29 16:05:18+00:00  
         14015 2020-08-29 16:45:57+00:00  
         18683 2020-08-29 17:15:19+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [55]:
store_matches_2 = {}

def find_related_crimes(C_Tar): # imagine the function taking in one row as its sole variable
    C_Tar['time_window_start'] = C_Tar['report_dt']
    C_Tar['time_window_end'] = C_Tar['report_dt'] + pd.Timedelta(minutes=time_window_minutes)

    merged = pd.merge(C_Tar, on='WARD', suffixes=('_tar'))

    matches = merged[(merged['report_dt_oth'] >= merged['time_window_start']) & 
                     (merged['report_dt_oth'] <= merged['time_window_end'])]

    # Select relevant columns and add matched CCN
    matches = matches[['CCN_tar', 'WARD', 'report_dt_oth']].rename(columns={'CCN_tar': 'matched_CCN'})
    
    return matches.reset_index(drop=True)

all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 734    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014      DAY  OTHERS         THEFT F/AUTO   
         14015      DAY  OTHERS         THEFT F/AUTO   
         18683      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 734         1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         9297        1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   
         14014       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         14015  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         18683       700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 734    ...       10700.0     Precinct 17  38.905648 -77.039822   
         9297   ...        5802.0    Precinct 129  38.897346 -77.027563   
         14014  ...        5303.0     Precinct 15  38.909638 -77.040089   
         14015  ...        4100.0     Precinct 13  38.913346 -77.050526   
         18683  ...        5801.0    Precinct 129  38.899121 -77.021926   

                            BID              START_DATE  \
20123250 734    GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         9297          DOWNTOWN  2020/08/29 23:08:57+00   
         14014              NaN  2020/08/28 22:00:23+00   
         14015              NaN  2020/08/26 22:00:29+00   
         18683         DOWNTOWN  2020/08/29 16:05:40+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 734    2020/08/29 19:00:05+00  596095073             NaN   
         9297                      NaN  596376108             NaN   
         14014  2020/08/29 08:00:27+00  596397251             NaN   
         14015  2020/08/27 12:00:51+00  596397252             NaN   
         18683  2020/08/29 16:08:33+00  596447637             NaN   

                               report_dt  
20123250 734   2020-08-29 22:04:46+00:00  
         9297  2020-08-30 00:05:52+00:00  
         14014 2020-08-29 16:05:18+00:00  
         14015 2020-08-29 16:45:57+00:00  
         18683 2020-08-29 17:15:19+00:00  

[5 rows x 26 columns]

### 2.2 apply it to one of the focal crimes

In [57]:
df1 = df[df.WARD==1]
df1["boop"]=1

/var/folders/2d/9d8wknmx1xdb64c0myhx8fl40000gn/T/ipykernel_35816/436923265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["boop"]=1


In [69]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here
    same_ward=C_Oth[ C_Oth['WARD']==r['WARD'] ]
    same_ward_same_time=same_ward[ (same_ward['report_dt']>=r['report_dt']) &
                                    (same_ward['report_dt']<=(r['report_dt']+timedelta(minutes=1200)))].copy()
    #the previous same_ward_same_time is not a new dataframe, but a slice/view of the original dataframe. 

    store_matches_2[str(r['CCN'])]=same_ward_same_time

r=C_Tar.iloc[0]

find_related_crimes(r)
store_matches_2

{'20165648':                X          Y       CCN              REPORT_DAT     SHIFT  \
 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   EVENING   
 1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00  MIDNIGHT   
 4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00       DAY   
 11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   EVENING   
 15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00       DAY   
 15617 -76.997316  38.904969  20165803  2020/11/20 14:45:06+00       DAY   
 16271 -76.994365  38.900203  20165859  2020/11/20 15:37:59+00       DAY   
 25170 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00       DAY   
 
        METHOD              OFFENSE  \
 279    OTHERS  MOTOR VEHICLE THEFT   
 1308   OTHERS  MOTOR VEHICLE THEFT   
 4675   OTHERS         THEFT F/AUTO   
 11621  OTHERS          THEFT/OTHER   
 15616  OTHERS          THEFT/OTHER   
 15617  OTHERS         THEFT F/AUTO   
 16271  OTHERS          THEFT

In [ ]:
df1

In [ ]:
for i,r in C_Tar.iterrows():
    same_ward=C_Oth[ C_Oth['WARD']==r['WARD'] ]
    same_ward_same_time=same_ward[ (same_ward['report_dt']>=r['report_dt']) &
                                    (same_ward['report_dt']<=(r['report_dt']+timedelta(minutes=1200)))

    store_matches_2[str(r['CCN'])]=same_ward_same_time


In [ ]:
#2.3 Use Apply to Cover all the other focal crimes